# goal:  fetch the minimal set of data from SPP sources for the current interval or a past interval
# persist in a database (try postgresql)


 ## TODO
 - DONE: figure out if we are ON or OFF the renaming of columns bandwagon.  We are ON the rename bandwagon for now, but a change to standardize_columns would change that.
     Background: https://dba.stackexchange.com/questions/250943/should-i-not-use-camelcase-in-my-column-names
 
 - REFACTOR to get the most recent available RTBM file, and use that as a time basis. 
     * Combine as much as possible in a common function 
     * Backfill: Provide the ability to query recent intervals that are not in the database, create a valid path from interval timestamps (stlf appears tricky), and slowly backfill a day of data
     * Optional Forward fill: for Multi-Day Resource Assessment, provide a way to forward-fill for a few days
 - CLEAN UP so that when exported to a python file it just runs without edits, and produces sane reports from each module

### Prerequisite:  
* miniconda installation from 2023 or later 
* in anaconda powershell prompt, "conda activate 2023"

In [2]:
#!pip install duckdb
import pandas as pd 

from datetime import datetime
import pytz


Source data model is in https://docs.google.com/spreadsheets/d/1Qh28Lb4dcbw9YMqcXLSj7N8l6Tlr46xNQkV-t1A2txc/edit#gid=0


## Define local database. Duckdb is cool and all, but how about a real database? 

  
    
    to initiate session:
    SET default_tablespace = u02_pgdata;
    create schema if not exists sppdata authorization current_user;
    set search_path to sppdata;
    
    to drop everything: 
     select 'drop table ' || table_name || ' cascade;' from information_schema.tables where table_schema = 'sppdata';



In [1]:
import json
# read the database information from the json file
with open('../dbconn.json', 'r') as f:
    di = json.load(f)
# create a connection string for postgresql
pg_uri = f"//{di['username']}:{di['password']}@{di['host']}:{di['port']}/{di['database']}"    


In [3]:
#!pip install sqlalchemy
#!pip install psycopg2

# using https://pythontic.com/pandas/serialization/postgresql as example
# though https://naysan.ca/2020/05/31/postgresql-to-pandas/ avoids the sqlalchemy layer
# Example python program to read data from a PostgreSQL table
# and load into a pandas DataFrame
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

# Create an engine instance
alchemyEngine   = create_engine(f'postgresql+psycopg2://{pg_uri}', pool_recycle=3600);

 # Connect to PostgreSQL server
con    = alchemyEngine.connect();
# con.execute (text("SET default_tablespace = u02_pgdata"))
# con.execute (text("create schema if not exists sppdata authorization current_user"))
# con.execute (text("set search_path to sppdata"))

con.autocommit=False;
# Read data from PostgreSQL database table and load into a DataFrame instance
#dataFrame       = pd.read_sql(text("select * from information_schema.tables"), con);
#dataFrame
 

def pgsqldf(query): 
    return pd.read_sql(text(query), con)


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

### example dataframe to table 

    dataFrame.to_sql("test_table", con=con, if_exists='replace', index=False);

### example table to dataframe: 
    df=pd.read_sql(text("select * from information_schema.tables"), con)

In [4]:
con.commit();  #  OH YEAH!  I'm on a real database!  changes need to be committed.


In [5]:
# if wanting to nuke the world and start over: 
if False: 
    for table_name in ['rtbm_lmp_by_location', 
        'da_lmp_by_location', 
        'area_control_error', 
        'stlf_vs_actual', 
        'mtlf_vs_actual', 
        'tie_flows_long', 
        'rtbm_binding_constraints',
        'generation_mix',
        'settlement_location',
    ]: 
        con.execute(text(f"drop table {table_name} cascade"))
 

In [6]:
# define a function to transform source data column names to a more appropriate form for working with data:
def standardize_columns(df): 
    df.columns = (df.columns
                    .str.replace('^ ', '', regex=True)
                    .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                    .str.replace('[_ ]+', '_', regex=True)
                    .str.lower()
                 )    
    
    # add an inserted time to all dataframes
    if not 'inserted_time' in df.columns.values: 
        df['inserted_time'] = datetime.now(pytz.timezone("America/Chicago"))

     
    print("standardized columns: ",  df.columns.values)

# Settlement Locations
and their estimated locations, from a local project file

In [7]:
if False: 
    # todo:  store the notebook that created this file and this file in documentation somewhere
    df=pd.read_csv("c:/Users/Operator/Downloads/UALR_MastersProject/SPPLocation/settlement_node_location.csv")

    """":# for database operations, rename all table and field names to lowercase snake_case
    df.rename(columns={'Settlement Location':'settlement_location',
               'InferredLocationType':'inferred_location_type',
               'est_Latitude':'est_latitude',
               'est_Longitude':'est_longitude'}, inplace=True)
    """

    standardize_columns(df)
    print(df.columns)

standardized columns:  ['settlement_location' 'inferred_location_type' 'est_latitude'
 'est_longitude' 'inserted_time']
Index(['settlement_location', 'inferred_location_type', 'est_latitude',
       'est_longitude', 'inserted_time'],
      dtype='object')


    # per https://duckdb.org/docs/guides/python/import_pandas.html, duckdb just knows about dataframes; no import necessary
    # create the table "my_table" from the DataFrame "my_df"
    duckdb.sql("CREATE or replace TABLE settlement_location AS SELECT * FROM df")
    # or
    # insert into the table "my_table" from the DataFrame "my_df"
    # duckdb.sql("INSERT INTO my_table SELECT * FROM my_df")

    # check that it got there
    duckdb.sql("SELECT table_catalog, table_name, column_name, is_nullable, data_type from information_schema.columns")

In [8]:
if False:    # this only needs to be done when settlement locations table changes, and now views depend on it.

    df.to_sql("settlement_location", con=con, if_exists='replace', index=False); 

    con.execute(text("""alter table settlement_location 
    add constraint settlement_location_pk 
    primary key (settlement_location)"""
                    )
               )

    con.commit()  #  OH YEAH!  I'm on a real database!  changes need to be committed.
  

In [9]:
# select sample data for fun
pd.read_sql(text("SELECT * from settlement_location order by random() limit 10"), con)

,settlement_location,inferred_location_type,est_latitude,est_longitude,inserted_time
0,GSPWR,Resource,35.244014,-102.683243,2023-03-02 19:14:47.084471+00:00
1,WAUE.NWPS.BEETHOVEN,Resource,43.185846,-98.075775,2023-03-02 19:14:47.084471+00:00
2,SPS.ENGCARBON1,Resource,35.691881,-101.407891,2023-03-02 19:14:47.084471+00:00
3,NPPD_HAST_WEC1,Resource,40.584211,-98.302139,2023-03-02 19:14:47.084471+00:00
4,EDE_MWW,Resource,38.602881,-95.906138,2023-03-02 19:14:47.084471+00:00
5,WR.WOLF,Resource,38.251850,-95.715891,2023-03-02 19:14:47.084471+00:00
6,WR_FLATRIDGESOUTH,Resource,37.373234,-98.250795,2023-03-02 19:14:47.084471+00:00
7,SECI.RUBART.RT04,Resource,37.556793,-101.066421,2023-03-02 19:14:47.084471+00:00
8,OMPA_LAMB_1,Resource,36.807890,-97.112764,2023-03-02 19:14:47.084471+00:00
9,WR_KP_CCPUCIC1,Resource,38.543348,-96.139474,2023-03-02 19:14:47.084471+00:00


### todo: Bug: if first append works but primary key fails to create, nothing else will work ever.

In [10]:
def pg_insertnew(table_name, primary_keys, df, con):
    # insert df into table_name but only if those rows aren't already there
    
    # make sure the target table exists, but empty (by iloc[0:0])
    try: 
        print ("pg_insertnew: trying append", table_name, con)
        con.commit() # maybe it needs no outstanding transactions before a DDL?  I think that was it
        df.to_sql(table_name, con=con, if_exists='append', index=False); 
        try_merge=False
    except: 
        print ("pg_insertnew append failed, rolling back")
        con.rollback();
        try_merge=True

        print ("pg_insertnew making sure table has PK")

    # make sure the target table has a primary key(s)
    con.execute(text(f"""
    DO $$
    BEGIN
        if NOT exists (
          select constraint_name
          from information_schema.table_constraints
          where table_name = '{table_name}' 
          and constraint_type = 'PRIMARY KEY'
        ) then ALTER TABLE {table_name}
          ADD CONSTRAINT {table_name}_pk PRIMARY KEY ({','.join(primary_keys)});
        end if;
    end $$"""))
    
    con.commit();

    if try_merge: 
#        print ("trying merge")
        print (f"pg_insertnew loading {table_name}_stg")
        # load df to a stage table
        df.to_sql(f"{table_name}_stg", con=con, if_exists='replace', index=False); 
        con.commit();

        # Insert new rows into permanent table
        joinstring=''
        for k in primary_keys: 
            if k == primary_keys[0]: 
                joinstring += f's.{k} = g.{k}'
            else:
                joinstring += f' and s.{k} = g.{k}'

        print (f"pg_insertnew loading {table_name}_stg to {table_name}")
        con.execute(text(f"""
           insert into {table_name}
           select s.* from {table_name}_stg s
           left join {table_name} g
           on {joinstring}
           where g.{primary_keys[0]} IS NULL
        """))

        con.commit();
        con.execute (text(f"""drop table {table_name}_stg"""))
        con.commit();    
                     
    return True

# Generation Mix

## todo: handle web server errors like 
- IncompleteRead: IncompleteRead(4044 bytes read, 2 more expected)


In [11]:
# try something harder: 2 hour generation mix. 

def update_generation_mix(con):
    df=pd.read_csv("https://marketplace.spp.org/file-browser-api/download/generation-mix-historical?path=%2FGenMix2Hour.csv", 
                   parse_dates=['GMT MKT Interval'], 
                   infer_datetime_format = True)
   
    standardize_columns(df)

    pg_insertnew(table_name='generation_mix', primary_keys=['gmt_mkt_interval'], df=df, con=con)
    
    return pgsqldf("select * from generation_mix order by gmt_mkt_interval desc limit 5")

# update_generation_mix(con)

standardized columns:  ['gmt_mkt_interval' 'coal_market' 'coal_self' 'diesel_fuel_oil_market'
 'diesel_fuel_oil_self' 'hydro_market' 'hydro_self' 'natural_gas_market'
 'natural_gas_self' 'nuclear_market' 'nuclear_self' 'solar_market'
 'solar_self' 'waste_disposal_services_market'
 'waste_disposal_services_self' 'wind_market' 'wind_self'
 'waste_heat_market' 'waste_heat_self' 'other_market' 'other_self' 'load'
 'inserted_time']
pg_insertnew: trying append generation_mix <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>
pg_insertnew append failed, rolling back
pg_insertnew making sure table has PK
pg_insertnew loading generation_mix_stg
pg_insertnew loading generation_mix_stg to generation_mix


,gmt_mkt_interval,coal_market,coal_self,diesel_fuel_oil_market,diesel_fuel_oil_self,hydro_market,hydro_self,natural_gas_market,natural_gas_self,nuclear_market,...,waste_disposal_services_market,waste_disposal_services_self,wind_market,wind_self,waste_heat_market,waste_heat_self,other_market,other_self,load,inserted_time
0,2023-03-08 22:05:00+00:00,7585.7,5122.1,5.0,0.0,187.9,936.9,9757.3,1006.0,0.0,...,1.5,7.3,0.0,3645.2,0.0,0.0,0.0,29.4,30763.075,2023-03-08 22:14:48.725867+00:00
1,2023-03-08 22:00:00+00:00,7513.3,5151.5,5.0,0.0,187.8,936.4,9723.2,1005.9,0.0,...,1.5,7.2,0.0,3646.4,0.0,0.0,0.0,29.6,30704.117,2023-03-08 22:14:48.725867+00:00
2,2023-03-08 21:55:00+00:00,7472.3,5159.1,5.0,0.0,249.5,902.5,9823.9,1004.5,0.0,...,1.4,7.3,0.0,3585.3,0.0,0.0,0.0,29.6,30703.708,2023-03-08 22:14:48.725867+00:00
3,2023-03-08 21:50:00+00:00,7399.0,5169.9,5.0,0.0,249.6,900.9,9754.6,1016.1,0.0,...,2.8,7.3,0.0,3610.6,0.0,0.0,0.0,29.6,30673.699,2023-03-08 22:14:48.725867+00:00
4,2023-03-08 21:45:00+00:00,7348.7,5178.4,5.0,0.0,249.5,902.8,9670.3,1027.6,0.0,...,2.9,7.5,0.0,3638.1,0.0,0.0,0.0,29.4,30609.458,2023-03-08 22:14:48.725867+00:00


In [12]:
# test query, just because I can:

pgsqldf("""
    select gmt_mkt_interval, 
     gmt_mkt_interval at time zone 'America/Chicago' as local_interval,
    coal_market+coal_self as coal,
    diesel_fuel_oil_market+diesel_fuel_oil_self as diesel,
    hydro_market+hydro_self as hydro,
    natural_gas_market+natural_gas_self as natural_gas,
    nuclear_market+nuclear_self as nuclear,
    solar_market+solar_self as solar,
    wind_market+wind_self as wind,
    waste_disposal_services_market+waste_disposal_services_self
      +waste_heat_market+waste_heat_self
      +other_market+other_self as other,
    load 
    from generation_mix
    order by gmt_mkt_interval desc
    limit 10
""")

,gmt_mkt_interval,local_interval,coal,diesel,hydro,natural_gas,nuclear,solar,wind,other,load
0,2023-03-08 22:05:00+00:00,2023-03-08 16:05:00,12707.8,5.0,1124.8,10763.3,2026.8,147.0,3645.2,38.2,30763.075
1,2023-03-08 22:00:00+00:00,2023-03-08 16:00:00,12664.8,5.0,1124.2,10729.1,2028.1,170.3,3646.4,38.3,30704.117
2,2023-03-08 21:55:00+00:00,2023-03-08 15:55:00,12631.4,5.0,1152.0,10828.4,2027.7,151.6,3585.3,38.3,30703.708
3,2023-03-08 21:50:00+00:00,2023-03-08 15:50:00,12568.9,5.0,1150.5,10770.7,2026.4,159.5,3610.6,39.7,30673.699
4,2023-03-08 21:45:00+00:00,2023-03-08 15:45:00,12527.1,5.0,1152.3,10697.9,2028.1,171.2,3638.1,39.8,30609.458
5,2023-03-08 21:40:00+00:00,2023-03-08 15:40:00,12475.4,5.0,1150.1,10699.0,2028.5,180.6,3651.2,39.2,30584.048
6,2023-03-08 21:35:00+00:00,2023-03-08 15:35:00,12453.6,5.0,1151.0,10727.3,2029.1,182.9,3633.8,39.2,30584.938
7,2023-03-08 21:30:00+00:00,2023-03-08 15:30:00,12427.7,5.0,1151.3,10764.2,2026.3,188.8,3658.7,39.1,30618.527
8,2023-03-08 21:25:00+00:00,2023-03-08 15:25:00,12408.2,5.0,1152.6,10778.7,2028.1,190.2,3665.8,38.9,30631.173
9,2023-03-08 21:20:00+00:00,2023-03-08 15:20:00,12383.6,5.0,1151.6,10836.4,2027.2,184.1,3656.6,38.9,30637.417


# RTBM LMP by Settlement Location

Depends on:  
generation_mix table, for the most recent interval. 
Will be stored in the ci (current interval) dataframe.

### TODO
 * determine if file names change with DST, and what the duplicate hour in November looks like
 * switch to grabbing that "latest interval" file
     * on FTP at pubftp.spp.org/Markets/RTBM/LMP_By_SETTLEMENT_LOC/RTBM-LMP-SL-latestInterval.csv 
     * on HTTPS at https://marketplace.spp.org/file-browser-api/download/rtbm-lmp-by-location?path=%2FRTBM-LMP-SL-latestInterval.csv
 



In [13]:
def get_current_interval(): 
    retdf = pgsqldf("""
    with c as (
        select max(gmt_mkt_interval) at time zone 'America/Chicago' as interval_cpt
        from generation_mix
    )
    , intervalmunge as (
        select interval_cpt, 
        '1970-01-01 00:00:00'::timestamp + (interval '5 minutes' * (floor(extract(EPOCH from interval_cpt)::numeric / 300.0) + 1)) as interval_end_cpt,
        '1970-01-01 00:00:00'::timestamp + (interval '1 hour' * (floor(extract(EPOCH from interval_cpt)::numeric / 3600.0) + 1)) as hour_end_cpt,
        '1970-01-01 00:00:00'::timestamp + (interval '1 hour' * (floor(extract(EPOCH from (interval_cpt + interval '5 minutes'))::numeric / 3600.0) + 1)) as pathhour_end_cpt  
        from c
    )
    select 
/*     date_part('year', interval_end_cpt)::char as rt_yyyy,
    lpad(date_part('month', interval_end_cpt)::char, 2, '0') as rt_mm,
    lpad(date_part('day', interval_end_cpt)::char, 2, '0') as rt_dd,
    lpad(date_part('hour', interval_end_cpt)::char, 2, '0') as rt_hh24,
    lpad(date_part('minute', interval_end_cpt)::char, 2, '0') as rt_mi,
*/
    to_char(interval_end_cpt, 'YYYY') as rt_yyyy,
    to_char(interval_end_cpt, 'MM') as rt_mm,
    to_char(interval_end_cpt, 'DD') as rt_dd,
    to_char(interval_end_cpt, 'HH24') as rt_hh24,
    to_char(interval_end_cpt, 'MI') as rt_mi,
    
    to_char(hour_end_cpt, 'YYYY') as da_yyyy,
    to_char(hour_end_cpt, 'MM') as da_mm,
    to_char(hour_end_cpt, 'DD') as da_dd,
    to_char(hour_end_cpt, 'HH24') as da_hh24,
    to_char(pathhour_end_cpt, 'HH24') as pathda_hh24
    
    from intervalmunge
    """)
    return retdf

con.rollback()
get_current_interval()


,rt_yyyy,rt_mm,rt_dd,rt_hh24,rt_mi,da_yyyy,da_mm,da_dd,da_hh24,pathda_hh24
0,2023,03,08,16,10,2023,03,08,17,17


In [14]:
def update_rtbm_lmp(con):
    # Pull out of generation_mix the most recent interval, in a format needed to get other information: 
    ci = get_current_interval()
    
    rt_yyyy=ci.rt_yyyy.values[0]
    rt_mm  =ci.rt_mm.values[0]
    rt_dd  =ci.rt_dd.values[0]
    rt_hh24  =ci.rt_hh24.values[0]
    rt_mi  =ci.rt_mi.values[0]

    # if this interval exists already in rtbm_lmp_by_location, skip the rest
    try: 
        rtbm_db_df=pgsqldf(f"""
        select *
        from rtbm_lmp_by_location
        where (gmtinterval_end at time zone 'America/Chicago') = '{rt_yyyy}-{rt_mm}-{rt_dd} {rt_hh24}:{rt_mi}:00'
        limit 5
        """)

        assert len(rtbm_db_df.index) > 0

        return rtbm_db_df
            
    except: 
        get_rtbm=True

    print ("updating RTBM data from current interval:", get_rtbm)

    fpath=f"https://marketplace.spp.org/file-browser-api/download/rtbm-lmp-by-location?" + \
          f"path=%2F{rt_yyyy}%2F{rt_mm}%2FBy_Interval%2F{rt_dd}%2F" + \
          f"RTBM-LMP-SL-{rt_yyyy}{rt_mm}{rt_dd}{rt_hh24}{rt_mi}.csv"

# todo:  handle 404 gracefully.  Sometimes the RTBM doesn't solve. 

    print (f"reading {fpath}")

    dfnew=pd.read_csv(fpath, parse_dates=['GMTIntervalEnd'], 
                   infer_datetime_format = True)

    """
    dfnew.rename(columns={'Interval':'interval', 'GMTIntervalEnd':'gmt_interval_end', 'Settlement Location':'settlement_location',
                   'Pnode':'pnode', 'LMP':'lmp', 'MLC':'mlc', 'MCC':'mcc', 'MEC':'mec'}, inplace=True)
    """ 
#   in this source file, GMTIntervalEnd does not have a timezone.  Add it: 
    from datetime import timezone
    dfnew['GMTIntervalEnd'] = dfnew['GMTIntervalEnd'].dt.tz_localize(timezone.utc)

    standardize_columns(dfnew)
    
# interval is now redundant 
    dfnew.drop(axis='columns', columns=['interval'], inplace=True)

#    return dfnew

    # insert rows that don't already exist
    
    pg_insertnew('rtbm_lmp_by_location', ['gmtinterval_end', 'settlement_location'], dfnew, con)
        
    con.commit()    
    
    rtbm_db_df=pgsqldf(f"""
       SELECT * 
       from rtbm_lmp_by_location 
       order by gmtinterval_end desc, random() limit 5
    """)
    return rtbm_db_df   

#con.rollback()
#update_rtbm_lmp(con)

updating RTBM data from current interval: True
reading https://marketplace.spp.org/file-browser-api/download/rtbm-lmp-by-location?path=%2F2023%2F03%2FBy_Interval%2F08%2FRTBM-LMP-SL-202303081610.csv
standardized columns:  ['interval' 'gmtinterval_end' 'settlement_location' 'pnode' 'lmp' 'mlc'
 'mcc' 'mec' 'inserted_time']
pg_insertnew: trying append rtbm_lmp_by_location <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>


,gmtinterval_end,settlement_location,pnode,lmp,mlc,mcc,mec,inserted_time
0,2023-03-08 22:10:00+00:00,OKGE.VOLT.0029,GLENDALE_DDR,74.8606,0.5066,0.0,74.3540,2023-03-08 22:14:50.394119+00:00
1,2023-03-08 22:10:00+00:00,SPS.VOLT.0045,SPSDOLLARHIUNDLHD_DDR_GEN_RA,77.2911,2.9371,0.0,74.3540,2023-03-08 22:14:50.394119+00:00
2,2023-03-08 22:10:00+00:00,WR.AEPE.FLATRIDGE3,WRVIOLA7UNFLATRDG3_AEPE_RA,73.4053,-0.9487,0.0,74.3540,2023-03-08 22:14:50.394119+00:00
3,2023-03-08 22:10:00+00:00,GSPR2015HUB,GSPR2015HUB_H,72.4690,-1.8850,0.0,74.3540,2023-03-08 22:14:50.394119+00:00
4,2023-03-08 22:10:00+00:00,GSPWR,SPSGSPWRUNGSPWR_WIND_RA,71.5397,-2.8144,0.0,74.3541,2023-03-08 22:14:50.394119+00:00


# DA LMP by Settlement Location



# TODO: fix the test in RTBM and DALMP that used 'interval' to determine if it needed to reload. 

In [15]:
def update_da_lmp(con):
    # Pull out of generation_mix the most recent interval, in a format needed to get other information: 
    ci = get_current_interval()
    
    da_yyyy=ci.da_yyyy.values[0]
    da_mm  =ci.da_mm.values[0]
    da_dd  =ci.da_dd.values[0]
    da_hh24  =ci.da_hh24.values[0]

        # if this interval exists already in da_lmp_by_location, skip the rest
    try: 
        da_db_df=pgsqldf(f"""
        select *
        from da_lmp_by_location
        where (gmtinterval_end at time zone 'America/Chicago') = '{da_yyyy}-{da_mm}-{da_dd} {da_hh24}:00:00'
        limit 5
        """)

        assert len(da_db_df.index) > 0

        return da_db_df
    except: 
        get_da=True

    print ("updating DA data from current interval:", get_da)

    fpath=f"https://marketplace.spp.org/file-browser-api/download/da-lmp-by-location?" + \
          f"path=%2F{da_yyyy}%2F{da_mm}%2FBy_Day%2FDA-LMP-SL-{da_yyyy}{da_mm}{da_dd}0100.csv"
        
    print (f"reading {fpath}")

    try: 
        # these are big; if I've already run once today it is cached
        dfnew=pd.read_pickle(f"DA-LMP-SL-{da_yyyy}{da_mm}{da_dd}0100.pickle")
        print (f"read local cached version DA-LMP-SL-{da_yyyy}{da_mm}{da_dd}0100.pickle")
    except: 
        dfnew=pd.read_csv(fpath, parse_dates=['GMTIntervalEnd'], 
                   infer_datetime_format = True)
        dfnew.to_pickle(f"DA-LMP-SL-{da_yyyy}{da_mm}{da_dd}0100.pickle")
        print (f"saved local cached version DA-LMP-SL-{da_yyyy}{da_mm}{da_dd}0100.pickle")
    """
    dfnew.rename(columns={'Interval':'interval', 'GMTIntervalEnd':'gmt_interval_end', 'Settlement Location':'settlement_location',
                   'Pnode':'pnode', 'LMP':'lmp', 'MLC':'mlc', 'MCC':'mcc', 'MEC':'mec'}, inplace=True)
    """
    
    #   in this source file, GMTIntervalEnd does not have a timezone.  Add it: 
    from datetime import timezone
    dfnew['GMTIntervalEnd'] = dfnew['GMTIntervalEnd'].dt.tz_localize(timezone.utc)

    standardize_columns(dfnew)
# interval is now redundant 
    dfnew.drop(axis='columns', columns=['interval'], inplace=True)

    
    # insert rows that don't already exist
    pg_insertnew('da_lmp_by_location', ['gmtinterval_end', 'settlement_location'], dfnew, con)
        
    con.commit()    
    
    da_db_df=pgsqldf(f"""
       SELECT * 
       from da_lmp_by_location 
       order by gmtinterval_end desc, random() limit 5
       """)
    return da_db_df   

#con.rollback()
#update_da_lmp(con)

,gmtinterval_end,settlement_location,pnode,lmp,mlc,mcc,mec,inserted_time
0,2023-03-08 23:00:00+00:00,AEC,SOUC,36.4822,0.8150,1.9102,33.757,2023-03-08 06:07:26.240155+00:00
1,2023-03-08 23:00:00+00:00,AECC_CSWS,CSWS_AECC_LA,35.1323,0.9239,0.4514,33.757,2023-03-08 06:07:26.240155+00:00
2,2023-03-08 23:00:00+00:00,AECC_ELKINS,CSWSELKINSUNELKINS_RA,34.8436,0.9876,0.0990,33.757,2023-03-08 06:07:26.240155+00:00
3,2023-03-08 23:00:00+00:00,AECC_FITZHUGH,CSWSFITZHUGHPLT1,35.6052,1.1357,0.7125,33.757,2023-03-08 06:07:26.240155+00:00
4,2023-03-08 23:00:00+00:00,AECC_FLTCREEK,CSWSFLINTCRKUN1_JOU_AECC_RA,34.5853,0.6856,0.1427,33.757,2023-03-08 06:07:26.240155+00:00


# Area Control Error
    ftp://pubftp.spp.org/Operational_Data/ACE/ACE.csv

In [16]:
def update_ace(con):
    table_name="area_control_error"
    source_url="ftp://pubftp.spp.org/Operational_Data/ACE/ACE.csv"
    primary_keys=['gmttime']
    
    df=pd.read_csv(source_url, 
                   parse_dates=['GMTTime'], 
                   infer_datetime_format = True
                  )
    
    print (df.columns.values)
    
    # df.rename(columns={'GMTTime':'gmt_time', 'Value':'value'}, inplace=True)
    standardize_columns(df)  
    print(df.columns)
    
    pg_insertnew(table_name=table_name, primary_keys=primary_keys, df=df, con=con)
    
    con.commit()
    
    return pgsqldf(f"select * from {table_name} order by gmttime desc limit 5")

#update_ace(con)

['GMTTime' 'Value']
standardized columns:  ['gmttime' 'value' 'inserted_time']
Index(['gmttime', 'value', 'inserted_time'], dtype='object')
pg_insertnew: trying append area_control_error <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>
pg_insertnew append failed, rolling back
pg_insertnew making sure table has PK
pg_insertnew loading area_control_error_stg
pg_insertnew loading area_control_error_stg to area_control_error


,gmttime,value,inserted_time
0,2023-03-08 22:14:00+00:00,51.9567,2023-03-08 22:14:51.926166+00:00
1,2023-03-08 22:13:00+00:00,62.1099,2023-03-08 22:14:51.926166+00:00
2,2023-03-08 22:12:00+00:00,68.2761,2023-03-08 22:14:51.926166+00:00
3,2023-03-08 22:11:00+00:00,71.0122,2023-03-08 22:14:51.926166+00:00
4,2023-03-08 22:10:00+00:00,119.5381,2023-03-08 22:14:51.926166+00:00


# STLF vs. Actual
    https://marketplace.spp.org/file-browser-api/download/stlf-vs-actual?path=%2F2023%2F02%2F25%2F15%2FOP-STLF-202302251435.csv
    
## TODO:  this one is strange; is there a file with more data?  If not, 
## need to figure out how replace older rows with newer ones while keeping ones that have already aged out

DONE - maybe delete rows with NULLs in Actual column before inserting new values?  If done in one transaction 
    * already done in Tie Flows; just do that
- need to remove commits from the pg_insertnew to keep client from seeing missing data between delete and insert

- at 23:30, tried to read
https://marketplace.spp.org/file-browser-api/download/stlf-vs-actual?path=/2023/03/01/23/OP-STLF-202303012330.csv
- but it is not there; latest file is pubftp.spp.org/Operational_Data/STLF/2023/03/01/00/OP-STLF-202303012330.csv

## fix start-of-hour 404: 
## at about 11:03, this was 404: 
    #  https://marketplace.spp.org/file-browser-api/download/stlf-vs-actual?path=/2023/03/02/11/OP-STLF-202303021100.csv
    # found here
    #  https://marketplace.spp.org/file-browser-api/download/stlf-vs-actual?path=/2023/03/02/12/OP-STLF-202303021100.csv
    # so the value of da_hh24 in the path had already advanced, 5 minutes early
    

In [17]:
def update_stlf(con):
    
    # Pull out of generation_mix the most recent interval, in a format needed to get other information: 
    ci = get_current_interval()
    
    rt_yyyy=ci.rt_yyyy.values[0]
    da_yyyy=ci.da_yyyy.values[0]
    rt_mm  =ci.rt_mm.values[0]
    da_mm  =ci.da_mm.values[0]
    rt_dd  =ci.rt_dd.values[0]
    da_dd  =ci.da_dd.values[0]
    rt_hh24  =ci.rt_hh24.values[0]
    da_hh24  =ci.da_hh24.values[0]
    pathda_hh24  =ci.pathda_hh24.values[0]
    rt_mi  =ci.rt_mi.values[0]
    
    table_name="stlf_vs_actual"
    
    source_url=f"https://marketplace.spp.org/file-browser-api/download/stlf-vs-actual?" + \
               f"path=%2F{rt_yyyy}%2F{rt_mm}%2F{rt_dd}%2F{pathda_hh24}%2FOP-STLF-{rt_yyyy}{rt_mm}{rt_dd}{rt_hh24}{rt_mi}.csv"
    primary_keys=['gmtinterval_end']
    
    print ("reading", source_url)
    
    df=pd.read_csv(source_url, 
                   parse_dates=['GMTInterval'], 
                   infer_datetime_format = True
                  )

    # fix this one error - end was left off of this table's timestamp
    df.rename(columns={'GMTInterval':'GMTIntervalEnd'}, inplace=True)

    print (df.columns.values)
    print (df)
        
#   in this source file, GMTIntervalEnd does not have a timezone.  Add it: 
    from datetime import timezone
    df['GMTIntervalEnd'] = df['GMTIntervalEnd'].dt.tz_localize(timezone.utc)
        
    standardize_columns(df)  
# interval is now redundant 
    df.drop(axis='columns', columns=['interval'], inplace=True)

    con.commit();
    
    try:
        con.execute(text("""delete from stlf_vs_actual where actual is null""")); 
        con.commit(); 
    except: 
        con.rollback();

    pg_insertnew(table_name=table_name, primary_keys=primary_keys, df=df, con=con)
    
    con.commit()
        
    return pgsqldf(f"select * from {table_name} order by gmtinterval_end desc limit 5")


#update_stlf(con)

reading https://marketplace.spp.org/file-browser-api/download/stlf-vs-actual?path=%2F2023%2F03%2F08%2F17%2FOP-STLF-202303081610.csv
['Interval' 'GMTIntervalEnd' 'STLF' 'Actual']
              Interval      GMTIntervalEnd   STLF   Actual
0  03/08/2023 16:20:00 2023-03-08 22:20:00  30867      NaN
1  03/08/2023 16:15:00 2023-03-08 22:15:00  30840      NaN
2  03/08/2023 16:10:00 2023-03-08 22:10:00  30853  30851.0
3  03/08/2023 16:05:00 2023-03-08 22:05:00  30794  30842.0
standardized columns:  ['interval' 'gmtinterval_end' 'stlf' 'actual' 'inserted_time']
pg_insertnew: trying append stlf_vs_actual <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>


,gmtinterval_end,stlf,actual,inserted_time
0,2023-03-08 22:20:00+00:00,30867,NaN,2023-03-08 22:14:52.382245+00:00
1,2023-03-08 22:15:00+00:00,30840,NaN,2023-03-08 22:14:52.382245+00:00
2,2023-03-08 22:10:00+00:00,30853,30851.0,2023-03-08 22:14:52.382245+00:00
3,2023-03-08 22:05:00+00:00,30794,30842.0,2023-03-08 22:14:52.382245+00:00
4,2023-03-08 21:40:00+00:00,30675,30714.0,2023-03-08 21:43:25.533105+00:00


# MTLF Vs. Actual
    https://marketplace.spp.org/file-browser-api/download/mtlf-vs-actual?path=%2F2023%2F02%2F25%2FOP-MTLF-202302251600.csv
    
## Todo:  
DONE same thing as STLF re null values
- this is kind of big and doesn't update but once an hour; maybe cache the file?
    

In [18]:
def update_mtlf(con):
    
    # Pull out of generation_mix the most recent interval, in a format needed to get other information: 
    ci = get_current_interval()
    
    rt_yyyy=ci.rt_yyyy.values[0]
    da_yyyy=ci.da_yyyy.values[0]
    rt_mm  =ci.rt_mm.values[0]
    da_mm  =ci.da_mm.values[0]
    rt_dd  =ci.rt_dd.values[0]
    da_dd  =ci.da_dd.values[0]
    rt_hh24  =ci.rt_hh24.values[0]
    da_hh24  =ci.da_hh24.values[0]
    rt_mi  =ci.rt_mi.values[0]
    
    table_name="mtlf_vs_actual"
    source_url=f"https://marketplace.spp.org/file-browser-api/download/mtlf-vs-actual?" + \
               f"path=%2F{rt_yyyy}%2F{rt_mm}%2F{rt_dd}%2FOP-MTLF-{rt_yyyy}{rt_mm}{rt_dd}{rt_hh24}00.csv"
    primary_keys=['gmtinterval_end']
    
    
    # if this interval exists already in the database, don't do this update
    try: 
        test_df=pgsqldf(f"""
        select *
        from mtlf_vs_actual
        where gmtinterval_end = '{da_yyyy}-{da_mm}-{da_dd} {da_hh24}:00:00'
        and averaged_actual is NOT NULL
        limit 5
        """)

        assert len(test_df.index) > 0

        print (f"update_mltf: found '{da_yyyy}-{da_mm}-{da_dd} {da_hh24}:00:00' already in database")
        return test_df
            
    except: 
        print (f"update_mltf: gmtinterval_end '{da_yyyy}-{da_mm}-{da_dd} {da_hh24}:00:00' not yet in database")
        get_rtbm=True
    
    
    print ("reading", source_url)
    
    # this file is not huge but consider caching it locally instead of reading from remote 12 times an hour
    # or, better, test the database to see if we need to update it.
    
    df=pd.read_csv(source_url, 
                   parse_dates=['GMTIntervalEnd'], 
                   infer_datetime_format = True
                  )
        
    # df.rename(columns={'Interval':'interval', 'GMTIntervalEnd':'gmt_interval_end', 'MTLF':'mtlf', 'Averaged Actual':'averaged_actual'}, inplace=True)
    
    #   in this source file, GMTIntervalEnd does not have a timezone.  Add it: 
    from datetime import timezone
    df['GMTIntervalEnd'] = df['GMTIntervalEnd'].dt.tz_localize(timezone.utc)
    
    standardize_columns(df)  

    # interval is now redundant 
    df.drop(axis='columns', columns=['interval'], inplace=True)

    
    try:
        con.execute(text("""delete from mtlf_vs_actual where averaged_actual is null""")); 
        con.commit(); 
    except: 
        con.rollback();
        

    pg_insertnew(table_name=table_name, primary_keys=primary_keys, df=df, con=con)
    
    con.commit()
        
    return pgsqldf(f"select * from {table_name} where averaged_actual is not null order by gmtinterval_end desc limit 5")

#update_mtlf(con)



update_mltf: gmtinterval_end '2023-03-08 17:00:00' not yet in database
reading https://marketplace.spp.org/file-browser-api/download/mtlf-vs-actual?path=%2F2023%2F03%2F08%2FOP-MTLF-202303081600.csv
standardized columns:  ['interval' 'gmtinterval_end' 'mtlf' 'averaged_actual' 'inserted_time']
pg_insertnew: trying append mtlf_vs_actual <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>
pg_insertnew append failed, rolling back
pg_insertnew making sure table has PK
pg_insertnew loading mtlf_vs_actual_stg
pg_insertnew loading mtlf_vs_actual_stg to mtlf_vs_actual


,gmtinterval_end,mtlf,averaged_actual,inserted_time
0,2023-03-08 23:00:00+00:00,30535,30842.0,2023-03-08 22:14:52.798231+00:00
1,2023-03-08 22:00:00+00:00,30325,30712.0,2023-03-08 21:07:23.733153+00:00
2,2023-03-08 21:00:00+00:00,30482,30931.0,2023-03-08 20:07:23.210850+00:00
3,2023-03-08 20:00:00+00:00,30769,31070.0,2023-03-08 19:07:23.337555+00:00
4,2023-03-08 19:00:00+00:00,30940,31407.0,2023-03-08 18:07:26.134751+00:00


# Tie Flows
    ftp://pubftp.spp.org/Operational_Data/TIE_FLOW/TieFlows.csv
    
## TODO: 
- Implement periodic vacuum from all these deletes

- Completely refactor to convert wide-form data to long-form data.  That would avoid completely breaking this interface if an area is added, removed or renamed.
    * AND it would completely fix the delete problem by not inserting NULL values in the first place. 

In [19]:
## DEPRECATED fragile wide-form; if the layout changes, we have to redo everything.  See now update_tie_flows_long. 
def update_tie_flows(con):
    return; 

    table_name="tie_flows"
    source_url="ftp://pubftp.spp.org/Operational_Data/TIE_FLOW/TieFlows.csv"
    primary_keys=['gmttime']
    
    df=pd.read_csv(source_url, 
                   parse_dates=['GMTTime'], 
                   infer_datetime_format = True
                  )
    
    #df.rename(columns={'GMTTime':'gmt_time'}, inplace=True)
    standardize_columns(df)
    
    try:
        con.execute(text("""delete from tie_flows where spp_nsi is null"""))
        con.commit() 
    except: 
        con.rollback()
        
    pg_insertnew(table_name=table_name, primary_keys=primary_keys, df=df, con=con)
    
    con.commit()
    
    return pgsqldf(f"""select * from {table_name} where spp_nsi is not null order by gmttime desc limit 5""")




In [20]:
#con.rollback()

def update_tie_flows_long(con):
    table_name="tie_flows_long"
    source_url="ftp://pubftp.spp.org/Operational_Data/TIE_FLOW/TieFlows.csv"
    primary_keys=['gmttime', 'area']
    
    df=pd.read_csv(source_url, 
                   parse_dates=['GMTTime'], 
                   infer_datetime_format = True
                  )
    
    df = pd.melt(df, id_vars=['GMTTime'], ignore_index=True).dropna()
    
    df.rename(columns={'GMTTime':'gmttime', 'variable':'area', 'value':'mw'}, inplace=True)
    
    standardize_columns(df)  # also adds inserted_time
    
    # remove future values that will be replaced
    #try:
    con.execute(text(f"""
      delete from "{table_name}" where area = 'SPP NSI Future'
      and gmttime > current_timestamp
      """))
    con.commit() 
    #except: 
    #    con.rollback()
    
    pg_insertnew(table_name=table_name, primary_keys=primary_keys, df=df, con=con)
    
    con.commit()
    
    return pgsqldf(f"""select * from {table_name} 
    where gmttime between current_timestamp - interval '2 minutes' and current_timestamp + interval '2 minutes'
    order by random() limit 5""")


#df = update_tie_flows_long(con)
#con.commit()

#df

standardized columns:  ['gmttime' 'area' 'mw' 'inserted_time']
pg_insertnew: trying append tie_flows_long <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>


,gmttime,area,mw,inserted_time
0,2023-03-08 22:13:00+00:00,ALTW,-233.859120,2023-03-08 22:14:54.353067+00:00
1,2023-03-08 22:13:00+00:00,SPC,-10.981597,2023-03-08 22:14:54.353067+00:00
2,2023-03-08 22:16:00+00:00,SPP NSI Future,-675.000000,2023-03-08 22:14:54.353067+00:00
3,2023-03-08 22:14:00+00:00,MEC,-456.173798,2023-03-08 22:14:54.353067+00:00
4,2023-03-08 22:13:00+00:00,ERCOTE,37.599998,2023-03-08 22:14:54.353067+00:00


# Real-Time Binding Constraints
    https://marketplace.spp.org/file-browser-api/download/rtbm-binding-constraints?path=%2FRTBM-BC-latestInterval.csv
    
 ## TODO
 DONE: figure out if we are ON or OFF the renaming of columns bandwagon - we are currently ON

In [21]:
def update_rt_binding(con):
    table_name="rtbm_binding_constraints"
    source_url="https://marketplace.spp.org/file-browser-api/download/rtbm-binding-constraints?path=%2FRTBM-BC-latestInterval.csv"
    primary_keys=['gmtinterval_end', 'constraint_name']
    
    df=pd.read_csv(source_url, 
                   parse_dates=['GMTIntervalEnd'], 
                   infer_datetime_format = True
                  )
    
    #print (df[['GMTIntervalEnd','Constraint Name','Constraint Type','NERCID','Monitored Facility']])

    #   in this source file, GMTIntervalEnd does not have a timezone.  Add it: 
    from datetime import timezone
    df['GMTIntervalEnd'] = df['GMTIntervalEnd'].dt.tz_localize(timezone.utc)

    # RT binding constraints file has dupes; fix it here
    df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
    
    """ df.rename(columns={'Interval':'interval', 'GMTIntervalEnd':'gmt_interval_end', 
                       'Constraint Name':'constraint_name', 'Constraint Type':'constraint_type',
                       'NERCID':'nercid', 'TLR Level':'tlr_level', 'State':'state', 'Shadow Price':'shadow_price',
                       'Monitored Facility':'monitored_facility', 'Contingent Facility':'contingent_facility'}, inplace=True)
    """
    standardize_columns(df)
    
    # interval is now redundant 
    df.drop(axis='columns', columns=['interval'], inplace=True)

#    con.execute(text("""delete from rtbm_binding_constraints where "SPP NSI" is null""")); 
#    con.commit(); 
      
    pg_insertnew(table_name=table_name, primary_keys=primary_keys, df=df, con=con)
    
    con.commit()
    
    return pgsqldf(f"""select * from {table_name} order by gmtinterval_end desc limit 5""")

#update_rt_binding(con)

standardized columns:  ['interval' 'gmtinterval_end' 'constraint_name' 'constraint_type' 'nercid'
 'tlr_level' 'state' 'shadow_price' 'monitored_facility'
 'contingent_facility' 'inserted_time']
pg_insertnew: trying append rtbm_binding_constraints <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>


,gmtinterval_end,constraint_name,constraint_type,nercid,tlr_level,state,shadow_price,monitored_facility,contingent_facility,inserted_time
0,2023-03-08 22:15:00+00:00,TMP643_28177,M2M,28177,CME,ACTIVATED,0.0,LN OAHE - SULLYBT,WAUE:FTTHOMP CHAPELLE:345:2:13,2023-03-08 22:14:56.814235+00:00
1,2023-03-08 22:15:00+00:00,TMP625_28362,FG,28362,CME,ACTIVATED,0.0,LN JAY_HAWK - FRANKLN5,KCPL:CNTRVILL DAKOTA:161:1:8,2023-03-08 22:14:56.814235+00:00
2,2023-03-08 22:15:00+00:00,TMP500_27577,FG,27577,CME,ACTIVATED,0.0,LN CAR395 - CARTHAGE,LNCJ366-ASB3491A161EDE,2023-03-08 22:14:56.814235+00:00
3,2023-03-08 22:15:00+00:00,TMP498_28023,FG,28023,CME,ACTIVATED,0.0,LN SNAKECK - ALIANCE,NPPD:SCOTSBLF VICTRY_H:115:1:5,2023-03-08 22:14:56.814235+00:00
4,2023-03-08 22:15:00+00:00,TMP462_27381,FG,27381,CME,ACTIVATED,0.0,LN CIMARRON - CKHAL,OKGE:DRAPR1 CIMARRON:345:1:11,2023-03-08 22:14:56.814235+00:00


# DONE.  
### Below here is just calling it again to make sure that works.


In [22]:
if True: 
    update_generation_mix(con)
    update_ace(con)
    update_rtbm_lmp(con)
    update_da_lmp(con)
    update_stlf(con)
    update_mtlf(con)
    update_tie_flows_long(con)
    update_rt_binding(con)
    con.commit()

standardized columns:  ['gmt_mkt_interval' 'coal_market' 'coal_self' 'diesel_fuel_oil_market'
 'diesel_fuel_oil_self' 'hydro_market' 'hydro_self' 'natural_gas_market'
 'natural_gas_self' 'nuclear_market' 'nuclear_self' 'solar_market'
 'solar_self' 'waste_disposal_services_market'
 'waste_disposal_services_self' 'wind_market' 'wind_self'
 'waste_heat_market' 'waste_heat_self' 'other_market' 'other_self' 'load'
 'inserted_time']
pg_insertnew: trying append generation_mix <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>
pg_insertnew append failed, rolling back
pg_insertnew making sure table has PK
pg_insertnew loading generation_mix_stg
pg_insertnew loading generation_mix_stg to generation_mix
['GMTTime' 'Value']
standardized columns:  ['gmttime' 'value' 'inserted_time']
Index(['gmttime', 'value', 'inserted_time'], dtype='object')
pg_insertnew: trying append area_control_error <sqlalchemy.engine.base.Connection object at 0x000001AC81E12020>
pg_insertnew append failed, roll

In [23]:
from time import sleep
while False:
    sleep(300)
    try: 
        update_generation_mix(con)
        update_ace(con)
        update_rtbm_lmp(con)
        update_da_lmp(con)
        update_stlf(con)
        update_mtlf(con)
        update_tie_flows(con)
        update_rt_binding(con)
        con.commit()    
    except: 
        con.rollback()
        

In [24]:
con.commit()